<a href="https://colab.research.google.com/github/Contato-Goes/ML_Arvore_Decisao/blob/main/Arvore_decisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

dados = pd.read_csv('/content/drive/MyDrive/creditcard.csv')
dados.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [7]:
n_transacao = dados['Time'].count()
n_fraude = dados['Class'].sum()
n_normal = n_transacao - n_fraude
fraude_porc = (n_fraude / n_transacao) * 100
normal_porc = (n_normal / n_transacao) * 100

print('Número de transações: ', n_transacao)
print('Número de fraudes: ', n_fraude, round(fraude_porc, 2),'%')
print('Número de transações normais: ', n_normal, round(normal_porc, 2),'%')

Número de transações:  284807
Número de fraudes:  492 0.17 %
Número de transações normais:  284315 99.83 %


Como o dados estão desequilibrados pois temos menos de 1% de fraudes, nosso modelo de Maching learning iria ficar com dados ruins para treinamento e teste, por isso o metodo StratifiedShuffleSplit vai seprar os dados com valores diferentes para que o treinamento e teste sejam mais precisos

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit
#N_split vai gerar uma divisão de paramentro
#test_Size vai dizer qual o tamanho do volume de teste (10%)
#random_state é gera um valor aleatorio no teste
def  executar_validador(x,y):
  validador = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
  for treino_id, teste_id in validador.split(x,y):
    x_treino, x_teste = x[treino_id], x[teste_id]
    y_treino, y_teste = y[treino_id], y[teste_id]
  return x_treino, x_teste, y_treino, y_teste

In [9]:
%%time
from sklearn import tree

def executar_classificador(classifcador,x_treino, x_teste, y_treino):

  arvore = classifcador.fit(x_treino, y_treino)
  y_pred = arvore.predict(x_teste)
  return y_pred

CPU times: user 486 ms, sys: 58.8 ms, total: 545 ms
Wall time: 773 ms


In [10]:
import matplotlib.pyplot as plt

def salvar_arvore(classificador, nome):
  plt.figure(figsize=(200,100))
  tree.plot_tree(classificador, filled=True, fontsize=14)
  plt.savefig(nome)
  plt.close()

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def validar_arvore(y_teste, y_pred):
  confusao = confusion_matrix(y_teste, y_pred)
  taxa_acerto = accuracy_score(y_teste, y_pred)
  print('Taxa de acerto: ', taxa_acerto)
  print('Precisão: ', precision_score(y_teste, y_pred))
  print('Recall: ', recall_score(y_teste, y_pred))
  print('Matriz de confusão: ',)
  print(confusao)
  return taxa_acerto

In [12]:
#Execução do validador
x = dados.drop('Class', axis=1).values
y = dados['Class'].values
x_treino, x_teste, y_treino, y_teste = executar_validador(x,y)

In [13]:
#Execucção do classificador
classifcador_arvore_decisao = tree.DecisionTreeClassifier()
y_pred_arvore_decisao = executar_classificador(classifcador_arvore_decisao,x_treino, x_teste, y_treino)

In [ ]:
#criação da figura da arvore de decisão
#salvar_arvore(classifcador_arvore_decisao, 'arvore_decisao1.png')

A matris de confusão separa as classes e compara o numero de erro do texto com a classificação, segue exemplo


---


Matriz de Confusão:

Nº acertos transações Nº erros transações (sem fraude)

Nº erros transações  Nº acertos transações (com fraude)

---
Os acertos e erros ficam na diagonal

In [ ]:
#Validação da arvore
validar_arvore(y_teste, y_pred_arvore_decisao)

In [27]:
print(classifcador_arvore_decisao)
print(classifcador_arvore_decisao.get_depth())

DecisionTreeClassifier()
21


In [28]:
#Execucção do classificador (arvore só com 10 de fundura e sem randomização)
classifcador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=10, random_state=0)
y_pred_arvore_decisao = executar_classificador(classifcador_arvore_decisao,x_treino, x_teste, y_treino)

In [30]:
#Nova validação
validar_arvore(y_teste, y_pred_arvore_decisao)

Taxa de acerto:  0.9994733330992591
Precisão:  0.9473684210526315
Recall:  0.7346938775510204
Matriz de confusão: 
[[28430     2]
 [   13    36]]


0.9994733330992591

In [31]:
#Execucção do classificador (arvore só com 10 de fundura e sem randomização, e com no minino 10 coisas que esqueci o que era)
classifcador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=10, random_state=0, min_samples_leaf=10)
y_pred_arvore_decisao = executar_classificador(classifcador_arvore_decisao,x_treino, x_teste, y_treino)

In [32]:
#Nova Nova validação
validar_arvore(y_teste, y_pred_arvore_decisao)

Taxa de acerto:  0.9993679997191109
Precisão:  0.8604651162790697
Recall:  0.7551020408163265
Matriz de confusão: 
[[28426     6]
 [   12    37]]


0.9993679997191109

In [33]:
#Execucção do classificador (arvore só com 5 de fundura e sem randomização)
classifcador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=10, random_state=0)
y_pred_arvore_decisao = executar_classificador(classifcador_arvore_decisao,x_treino, x_teste, y_treino)

In [34]:
#Nova Nova Nova validação
validar_arvore(y_teste, y_pred_arvore_decisao)

Taxa de acerto:  0.9994733330992591
Precisão:  0.9473684210526315
Recall:  0.7346938775510204
Matriz de confusão: 
[[28430     2]
 [   13    36]]


0.9994733330992591